In [1]:
import swarmrl.engine.resobee as resobee
import os
infomsg = "I "

import copy
import tempfile
import unittest as ut

import flax.linen as nn
import numpy as np
import optax
import pint

import swarmrl as srl
import swarmrl.engine.espresso as espresso
from swarmrl.models.interaction_model import Action
from swarmrl.utils import utils

Could not find espressomd. Features will not be available


## RL Configuration

In [2]:
class ActorNet(nn.Module):
    """A simple dense model."""

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=128)(x)
        x = nn.relu(x)
        x = nn.Dense(features=4)(x)
        return x

class CriticNet(nn.Module):
    """A simple dense model."""

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=128)(x)
        x = nn.relu(x)
        x = nn.Dense(features=1)(x)
        return x

# Define networks
critic_stack = CriticNet()
actor_stack = ActorNet()

# Define an exploration policy
exploration_policy = srl.exploration_policies.RandomExploration(probability=0.1)

# Define a sampling_strategy
sampling_strategy = srl.sampling_strategies.GumbelDistribution()

# Value function to use
value_function = srl.value_functions.ExpectedReturns(
    gamma=0.99, standardize=True
)

# Define the models.
actor = srl.networks.FlaxModel(
    flax_model=actor_stack,
    optimizer=optax.adam(learning_rate=0.001),
    input_shape=(1,),
    sampling_strategy=sampling_strategy,
    exploration_policy=exploration_policy,
)
critic = srl.networks.FlaxModel(
    flax_model=critic_stack,
    optimizer=optax.adam(learning_rate=0.001),
    input_shape=(1,),
)

def scale_function(distance: float):
    """
    Scaling function for the task
    """
    return 1 / distance

task = srl.tasks.searching.GradientSensing(
    source=np.array([10.0, 10.0]),
    decay_function=scale_function,
    reward_scale_factor=1000,
    box_length=np.array([20.0, 20.]),
)

observable = srl.observables.ConcentrationField(
    source=np.array([10.0, 10.0]),
    decay_fn=scale_function,
    scale_factor=10000,
    box_length=np.array([20.0, 20.0]),
    particle_type=0,
)

# Define the loss model
loss = srl.losses.PolicyGradientLoss(value_function=value_function)
translate = Action(force=10.0)
rotate_clockwise = Action(torque=np.array([0.0, 0.0, 10.0]))
rotate_counter_clockwise = Action(torque=np.array([0.0, 0.0, -10.0]))
do_nothing = Action()

actions = {
    "TranslateLeft": np.array([-10., 0.]),
    "TranslateForward": np.array([0., 10.]),
    "TranslateRight": np.array([10., 0.]),
    "MoveBackwards": np.array([0., -10.]),
}
protocol = srl.rl_protocols.ActorCritic(
    particle_type=0,
    actor=actor,
    critic=critic,
    task=task,
    observable=observable,
    actions=actions,
)
# Define the force model.
rl_trainer = srl.gyms.Gym(
    [protocol],
    loss,
)

In [3]:
resobee_root_path = "/home/user/ResoBee/"

build_path = os.path.join(resobee_root_path, "build")
config_dir = os.path.join(resobee_root_path, 'workflow/projects/debug/parameter-combination-0/seed-0')

target = 'many_body_simulation'
resobee_executable = os.path.join(resobee_root_path, 'build/src', target)


In [4]:
system_runner = resobee.ResoBee(
    resobee_executable=resobee_executable,
    config_dir=config_dir
)

Binding to tcp address:  tcp://127.0.0.1:5544


In [5]:
# todo: cannot overwrite h5 output file for some reason
rl_trainer.perform_rl_training(
            system_runner=system_runner,
            n_episodes=500,
            episode_length=100,
        )
# system_runner.integrate(10, None)


Output()

The Many-Body Simulations and Machine Learning Group proudly presents...
▒▒░░  ▒▒░░░░░░░░░░░░░░░░░░░░░░░░██  ░░██  ░░  ▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░  
▒▒  ░░  ██░░░░░░░░░░░░░░░░░░░░░░  ░░    ██      ░░░░░░░░░░░░░░░░░░░░░░░░    ██
▓▓▓▓    ▒▒░░░░░░░░░░░░░░░░░░░░░░    ██▓▓▓▓░░  ▒▒░░░░░░░░░░░░░░░░░░░░░░▒▒  ░░▓▓
░░▒▒▓▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▓▓  ░░░░▓▓▒▒░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░
▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░▓▓░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░▓▓░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░░░████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒████░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░▒▒████▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓████▒▒░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░▓▓▒▒████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▓▓██▓▓▓▓░░░░░░░░░░░░░░
░░░░░░░░░░░░░░▒▒░░░░██  ▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒  ██▒▒░░▒▒░░░░░░░░░░░░
░░░░░░░░░░░░▒▒  ░░  ██░░  ▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░  ▒▒░░░░░░░░░░
░░░░░░░░░░██  ░░  ▓▓    ░░  ░░░░░░░░░░░░░░░░░░░░░░██  ░░░░

ResoBee simulation successfully completed.

/home/user/conda/envs/manybodyrc/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.py:2439: UserWarning: 
Explicitly requested dtype float64 requested in asarray is not available, and will be truncated to dtype float32. 
To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment 
variable. See https://github.com/google/jax#current-gotchas for more.
  start = asarray(start, dtype=computation_dtype)

/home/user/conda/envs/manybodyrc/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.py:2440: UserWarning: 
Explicitly requested dtype float64 requested in asarray is not available, and will be truncated to dtype float32. 
To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment 
variable. See https://github.com/google/jax#current-gotchas for more.
  stop = asarray(stop, dtype=computation_dtype)

...

The Many-Body Simulations and Machine Learning Group proudly presents...
▒▒░░  ▒▒░░░░░░░░░░░░░░░░░░░░░░░░██  ░░██  ░░  ▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░  
▒▒  ░░  ██░░░░░░░░░░░░░░░░░░░░░░  ░░    ██      ░░░░░░░░░░░░░░░░░░░░░░░░    ██
▓▓▓▓    ▒▒░░░░░░░░░░░░░░░░░░░░░░    ██▓▓▓▓░░  ▒▒░░░░░░░░░░░░░░░░░░░░░░▒▒  ░░▓▓
░░▒▒▓▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▓▓  ░░░░▓▓▒▒░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░
▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░▓▓░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░▓▓░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░░░████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒████░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░▒▒████▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓████▒▒░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░▓▓▒▒████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▓▓██▓▓▓▓░░░░░░░░░░░░░░
░░░░░░░░░░░░░░▒▒░░░░██  ▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒  ██▒▒░░▒▒░░░░░░░░░░░░
░░░░░░░░░░░░▒▒  ░░  ██░░  ▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░  ▒▒░░░░░░░░░░
░░░░░░░░░░██  ░░  ▓▓    ░░  ░░░░░░░░░░░░░░░░░░░░░░██  ░░░░

ResoBee simulation successfully completed.

The Many-Body Simulations and Machine Learning Group proudly presents...
▒▒░░  ▒▒░░░░░░░░░░░░░░░░░░░░░░░░██  ░░██  ░░  ▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░  
▒▒  ░░  ██░░░░░░░░░░░░░░░░░░░░░░  ░░    ██      ░░░░░░░░░░░░░░░░░░░░░░░░    ██
▓▓▓▓    ▒▒░░░░░░░░░░░░░░░░░░░░░░    ██▓▓▓▓░░  ▒▒░░░░░░░░░░░░░░░░░░░░░░▒▒  ░░▓▓
░░▒▒▓▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▓▓  ░░░░▓▓▒▒░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░
▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░▓▓░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░▓▓░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░░░████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒████░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░▒▒████▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓████▒▒░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░▓▓▒▒████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▓▓██▓▓▓▓░░░░░░░░░░░░░░
░░░░░░░░░░░░░░▒▒░░░░██  ▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒  ██▒▒░░▒▒░░░░░░░░░░░░
░░░░░░░░░░░░▒▒  ░░  ██░░  ▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░  ▒▒░░░░░░░░░░
░░░░░░░░░░██  ░░  ▓▓    ░░  ░░░░░░░░░░░░░░░░░░░░░░██  ░░░░

ResoBee simulation successfully completed.

The Many-Body Simulations and Machine Learning Group proudly presents...
▒▒░░  ▒▒░░░░░░░░░░░░░░░░░░░░░░░░██  ░░██  ░░  ▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░  
▒▒  ░░  ██░░░░░░░░░░░░░░░░░░░░░░  ░░    ██      ░░░░░░░░░░░░░░░░░░░░░░░░    ██
▓▓▓▓    ▒▒░░░░░░░░░░░░░░░░░░░░░░    ██▓▓▓▓░░  ▒▒░░░░░░░░░░░░░░░░░░░░░░▒▒  ░░▓▓
░░▒▒▓▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▓▓  ░░░░▓▓▒▒░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░
▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░▓▓░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░▓▓░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░░░████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒████░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░▒▒████▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓████▒▒░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░▓▓▒▒████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▓▓██▓▓▓▓░░░░░░░░░░░░░░
░░░░░░░░░░░░░░▒▒░░░░██  ▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒  ██▒▒░░▒▒░░░░░░░░░░░░
░░░░░░░░░░░░▒▒  ░░  ██░░  ▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░  ▒▒░░░░░░░░░░
░░░░░░░░░░██  ░░  ▓▓    ░░  ░░░░░░░░░░░░░░░░░░░░░░██  ░░░░